# Assignment Group 3
Name: AKhilesh Yadav  Gaddam

## Module C _(59 points)_

In this exercise we'll build and expand a cheese and wine pairings database with the high-level goal of linking wine varieties and recipe ingredients.

A Google search on food and wine pairings turned up a literature Ph.D. candidate's blog post:
+ http://sedimentality.com/drinking-wine/list-of-wine-and-food-pairings/

This post is interesting, because instead of pairing dishes with wine vintages or brands, it pairs ingredients with wine varieties, tagging ingredients by several categories. Thus, we could pull the data from this website to join wine varieties from a large wine reviews database to ingredients in a large recipes database. Your goal in this section of the assignment is to collect the ingredient-variety-category data.

__C1.__ _(2 pts)_ Why are we allowed to scrape this data out of the website?

Explain what information is posted (or not posted) on this domain, allowing us to definitely move forward with this exercise. Use the markdown cell below.

There are no terms for this website, so we can scrap the data easily without any permissions.

_Response._

__C2.__ _(2 pts)_ Download the blog post using the `requests` module and store the html response.

In [2]:
from bs4 import BeautifulSoup
import urllib
html_text = urllib.request.urlopen("http://sedimentality.com/drinking-wine/list-of-wine-and-food-pairings/").read()
soup = BeautifulSoup(html_text, 'html.parser')
print(soup)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en-US">
<head profile="http://gmpg.org/xfn/11">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="http://sedimentality.com/xmlrpc.php" rel="pingback"/>
<title>List of wine and food pairings | Sedimentality</title>
<!-- All in One SEO Pack 3.2.10 by Michael Torbert of Semper Fi Web Design[399,452] -->
<meta content="&quot;What wine goes with Captain Crunch?&quot; -George Karlin Pairing wines and food is too often stressed about (and debated). True, in these days we no longer abide by rules like &quot;ONLY white with fish&quot; (ever had a coriander and coffee encrusted ahi with Syrah? Amazing!) but we are often confused about how to pair certain flavors and elements of a dish. The following are some guidelines for pairing a few common wines. Remember, 

__C3.__ _(4 pts)_ Navigate to the same URL in your web browser, and identify where the target data are by inspecting the html (with your eyes) and matching up tags with the target content.

In the markdown cell below, describe the tags that you will need to extract the data from. Provide as much information as is necessary on the nesting of the document so as to be able to describe the location of the target data for parsing.

_Response._

Tag 'h2' contains all the different wines. The tags 'p' under each 'h2' tag has the data. 
I want to first get the data and later extract the data for wines as all the wine types 
have the same structure that is all of them has same number of food pairings.

__C4.__ _(6 pts)_ Parse the html text with BeautifulSoup in preparation for scraping. Then, print out the target data. \[__Hint__: use `BeautifulSoup`'s `.find_all(tag_list)` method to iterate through the tags identified in __C3__. From there, check to see if a tag has the data of interest by using the `.find(tag)` method. \]

In [3]:
from bs4 import BeautifulSoup
import re
category=soup.find_all('strong')
wine=soup.find_all('h2')
new=[]
for i in category:
    i=i.parent
    new.append(i)
    
for i in wine:
    print(i.text)

Sauvignon Blanc
Chardonnay
Riesling
Pinot Noir
Syrah
Merlot
Cabernet Sauvignon 


__C5.__ _(3 pts)_ Interpret the output from the previous part to make a plan for how to split the target data up with regular expressions to create a structured data object. Describe any structure that you'll use in the markdown cell, below.

It seems that every wine category has 7 food categories. 
This nested information can be held in a default dictionary.

_Response._

__C6.__ _(2 pts)_ Create an associative array (dictionary) data structure that can hold the ingredients in their categories, by wine variety. Use a default dictionary with its default value as a dictionary.

In [4]:
from collections import defaultdict

dictionary= defaultdict(dict)
print(dictionary)

defaultdict(<class 'dict'>, {})


__C7.__ _(7 pts)_ Complete the web-scraping script's loop in __C4__ and store the extracted data in the data structure you initialized in __C6__. Print the stored data to confirm your output. Note you'll need to use the regular expresions module to complete this section.

In [5]:
import re,json
from pprint import pprint

required=[]
for i in new:
    required.append(i.text)
temp=[]    
for i in required:
    if re.findall(r'\xa0',i):
        i=re.sub(r'\xa0',' ',i)
        
    temp.append(i)
    
for i in temp[:len(temp)-1]:
    index=temp.index(i)
    if temp[index]==temp[index+1]:
        temp.remove(i)

data=[temp[i:i + 8] for i in range(0, len(temp), 8)]

for i in data:
    wine=i[0]
    dictionary[wine]={}
    
    for j in i[1:]:
        sp=re.split(':',j)
        sp2=re.split(',',sp[1])
        dictionary[wine][sp[0]]=sp2


result=dict(dictionary)
pprint(result)

{'Cabernet Sauvignon ': {'Cheese/nuts': [' cheddar', ' gorgonzola', ' walnuts'],
                         'Desserts': [' bittersweet chocolate'],
                         'Fruits and Veggies': [' black cherries',
                                                ' tomatoes',
                                                ' broccoli'],
                         'Herbs and Spices': [' rosemary',
                                              ' juniper',
                                              ' lavender'],
                         'Meat/poultry': [' venison', ' rib eye', ' beef stew'],
                         'Sauces': [' brown sauce', ' tomato sauce'],
                         'Seafood': [' grilled ahi tuna']},
 'Chardonnay': {'Cheese/nuts': [' mild',
                                ' semi-soft cheeses with unoaked Chardonnay; '
                                'asiago',
                                ' havarti',
                                ' Stilton or other blue-veined cheeses

__C8.__ _(2 pts)_ Write your data out to a json file. Name this file: `./data/wine_ingredient-pairings.json`.

Note: since `defaultdicts` are not json serialiable (you can't make a file out of them), you will have to coerce (type cast) your final data to a standard Python dictionary to finish the job.

In [6]:
f=open('data/wine_ingredient-pairings.json','w')
json.dump(result,f)
f.close()

__C9.__ _(3 pts)_ What limitations did you observe in this data, and in your extraction of this data?

Were there any issues either with this data's content, volume, its veracity or accuracy? Were you able to get all of the data present? Did you wind up getting some data that you didn't want or need, or maybe not in the right form? Take a look at the wine and categories keys—are they consistent? Be as specific as possible!

_Response._

The data scraped wasn't that good. It would have been helpfull if the data was more appropriate, instead of content placed in child tag of the heading rather it would have been placed in sibling.
However the wine and category keys looks good.

__C10.__ _(2 pts)_ Supposing we were going to rebuild the scraped wine-food pairings data as a gold standard through a survey. Are there any changes to allowable food types or to the manner in which the data are organized that might be beneficial to the wine recommendation task?

_Response._

Ingredients which are of similar characteristics should have been used.

__C11.__ _(3 pts)_ Download and familiarize yourself with the cuisine prediction dataset: https://www.kaggle.com/kaggle/recipe-ingredients-dataset (it's only necessary to download the `train.json` file and work with that).
We're going to enrich our current data by adding results from this dataset.

In [7]:
import json
out=open('data/train.json')
f=out.read()
new=json.loads(f)
new

[{'id': 10259,
  'cuisine': 'greek',
  'ingredients': ['romaine lettuce',
   'black olives',
   'grape tomatoes',
   'garlic',
   'pepper',
   'purple onion',
   'seasoning',
   'garbanzo beans',
   'feta cheese crumbles']},
 {'id': 25693,
  'cuisine': 'southern_us',
  'ingredients': ['plain flour',
   'ground pepper',
   'salt',
   'tomatoes',
   'ground black pepper',
   'thyme',
   'eggs',
   'green tomatoes',
   'yellow corn meal',
   'milk',
   'vegetable oil']},
 {'id': 20130,
  'cuisine': 'filipino',
  'ingredients': ['eggs',
   'pepper',
   'salt',
   'mayonaise',
   'cooking oil',
   'green chilies',
   'grilled chicken breasts',
   'garlic powder',
   'yellow onion',
   'soy sauce',
   'butter',
   'chicken livers']},
 {'id': 22213,
  'cuisine': 'indian',
  'ingredients': ['water', 'vegetable oil', 'wheat', 'salt']},
 {'id': 13162,
  'cuisine': 'indian',
  'ingredients': ['black pepper',
   'shallots',
   'cornflour',
   'cayenne pepper',
   'onions',
   'garlic paste',
   'm

__C12.__ _(8 pts)_ Expand each wine variety's list of foods to include any co-appearing in the recipes found in the above Kaggle dataset. Make sure to record how often ingredients in the Kaggle recipes occured alongside foods in the wine variety list. Store this information in another nested `defaultdict`, with the following structure:

```
wines = {
    wine: {
        food_category: {
            pairing: Counter(),
            ...
        },
        ...
    },
    ...
}
```

In [8]:
ing=[]
for i in new:
    lis=i['ingredients']
    ing.append(lis)
    
ing

[['romaine lettuce',
  'black olives',
  'grape tomatoes',
  'garlic',
  'pepper',
  'purple onion',
  'seasoning',
  'garbanzo beans',
  'feta cheese crumbles'],
 ['plain flour',
  'ground pepper',
  'salt',
  'tomatoes',
  'ground black pepper',
  'thyme',
  'eggs',
  'green tomatoes',
  'yellow corn meal',
  'milk',
  'vegetable oil'],
 ['eggs',
  'pepper',
  'salt',
  'mayonaise',
  'cooking oil',
  'green chilies',
  'grilled chicken breasts',
  'garlic powder',
  'yellow onion',
  'soy sauce',
  'butter',
  'chicken livers'],
 ['water', 'vegetable oil', 'wheat', 'salt'],
 ['black pepper',
  'shallots',
  'cornflour',
  'cayenne pepper',
  'onions',
  'garlic paste',
  'milk',
  'butter',
  'salt',
  'lemon juice',
  'water',
  'chili powder',
  'passata',
  'oil',
  'ground cumin',
  'boneless chicken skinless thigh',
  'garam masala',
  'double cream',
  'natural yogurt',
  'bay leaf'],
 ['plain flour',
  'sugar',
  'butter',
  'eggs',
  'fresh ginger root',
  'salt',
  'ground 

In [9]:
from collections import Counter


wines={}
for i in result:
    temp={}
    for j in result[i]:
        temp2={}
        for k in result[i][j]:
            
            count= Counter()
            for l in ing:
                if k.strip() in l:
                    for m in l:
                        if k.strip()!=m:
                            count[m]+=1
            temp2[k]=count
        temp[j]=temp2
    wines[i]=temp
wines

{'Sauvignon Blanc': {'Cheese/nuts': {' feta': Counter(),
   ' goat cheese': Counter({'eggplant': 11,
            'fresh basil leaves': 6,
            'cooking spray': 18,
            'chicken': 1,
            'pitas': 1,
            'plum tomatoes': 12,
            'cherry tomatoes': 6,
            'baby spinach': 6,
            'sour cream': 7,
            'flour tortillas': 5,
            'tuna steaks': 1,
            'freshly ground pepper': 8,
            'olive oil': 65,
            'salt': 77,
            'eggs': 10,
            'russet potatoes': 1,
            'mexican chorizo': 2,
            'black beans': 11,
            'salsa': 7,
            'canola oil': 5,
            'colby cheese': 1,
            'black olives': 1,
            'avocado': 9,
            'half & half': 3,
            'low sodium chicken broth': 2,
            'pepper': 24,
            'grated parmesan cheese': 15,
            'zucchini': 7,
            'polenta': 4,
            'water': 16,
            

__C13.__ _(2 pts)_ How would you create the above structure for `wines` using nested a nested `defaultdict`?

nested = defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : Counter())))

__C14.__ _(6 pts)_ We'd also like to incorporate a wine review dataset. Download and familiarize yourself with the dataset, found at: https://www.kaggle.com/zynicide/wine-reviews. Specifically, utilize the file: `winemag-data-130k-v2.csv` to link each of the scraped wines found in `wines` to the specific names of the wines which are of that variety (i.e., for the Sauvignon wine, create a list of all the wines with `variety` column that say Sauvignon, and attach them to the `wines` object from before).

In [10]:
import pandas as pd
df=pd.read_csv('data/winemag-data-130k-v2.csv')
title=[]
variety=[]
for i in df['title']:
    title.append(i)
for i in df['variety']:
    variety.append(i)

In [11]:
t=list(zip(title,variety))

for i in wines:
    names=[]
    for j in t:
        if i==j[1]:
            names.append(j[0])
            
    wines[i]['names']=names

In [12]:
wines['Sauvignon Blanc']['names']

['Envolve 2011 Sauvignon Blanc (Sonoma Valley)',
 'Robert Hall 2011 Sauvignon Blanc (Paso Robles)',
 'Trump 2011 Sauvignon Blanc (Monticello)',
 'Sevtap 2015 Golden Horn Sauvignon Blanc (Santa Ynez Valley)',
 'Stoneleigh 2008 Sauvignon Blanc (Marlborough)',
 'Long Flat 2006 Destinations Sauvignon Blanc (Adelaide Hills)',
 'Elena Walch 2002 Castel Ringberg Sauvignon Blanc (Alto Adige)',
 'Borgo Conventi 2002 I Fiori del Borgo Sauvignon Blanc (Collio)',
 "Ceago Vinegarden 2001 Kathleen's Vineyard Sauvignon Blanc (Mendocino County)",
 'Vinoce 2001 Sauvignon Blanc (Rutherford)',
 'Murphy-Goode 2000 Reserve Fume Sauvignon Blanc (Alexander Valley)',
 'Bridlewood 2000 Sauvignon Blanc (Santa Ynez Valley)',
 'Aresti 2008 Estate Selection Sauvignon Blanc (Curicó Valley)',
 'Ojai 2014 McGinley Vineyard Sauvignon Blanc (Santa Ynez Valley)',
 'Hanna 2011 Sauvignon Blanc (Russian River Valley)',
 'Strauss 2016 Classic Sauvignon Blanc (Südsteiermark)',
 'Monte Xanic 2012 Viña Kristel Sauvignon Blanc 

__C15.__ _(5 pts)_ Now write a function which allows the user to input a list of ingredients and also a choice of ingredient category (such as "Cheese/nuts"), and outputs a list of all wine names that pair with these ingredients. This function won't incorporate the additional ingredients we obtained from the Kaggle dataset.

In [13]:
def func(ingredients, category):
    result=[]
    for i in wines:
        if category in [*wines[i]]:
            lis=[x.strip() for x in [*wines[i][category]]]
            if any(True for x in ingredients if x in lis):
                for k in wines[i]['names']:
                    result.append(k)
    return result
func(['Havarti','gouda'], 'Cheese/nust')

['St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)',
 'Richard Böcking 2013 Devon Riesling (Mosel)',
 'The White Knight 2011 Riesling (Lake County)',
 'Schmitt Söhne 2015 Riesling (Rheinhessen)',
 'P.J. Valckenberg 2015 Undone Dry Riesling (Rheinhessen)',
 'Heron Hill 2015 Ingle Vineyard Riesling (Finger Lakes)',
 'Lamoreaux Landing 2014 Red Oak Vineyard Riesling (Finger Lakes)',
 'Lamoreaux Landing 2014 Yellow Dog Vineyard Riesling (Finger Lakes)',
 'Dopff & Irion 2004 Schoenenbourg Grand Cru Vendanges Tardives Riesling (Alsace)',
 'Banyan 2007 Riesling (Santa Lucia Highlands)',
 'Weingut Liebfrauenstift 2014 Dry Riesling (Rheinhessen)',
 'Carl Graff 2014 Graacher Himmelreich Spätlese Riesling (Mosel)',
 'Claiborne & Churchill 2014 Claiborne Vineyard Riesling (Edna Valley)',
 'Fritz Haag 2014 Brauneberger Feinherb Riesling (Mosel)',
 'Fritz Haag 2014 Riesling (Mosel)',
 'Treleaven 2006 Semi-Dry Riesling (Cayuga Lake)',
 'Salomon-Undhof 2011 Steiner Kögl Erste Lage R

__C16.__ _(2 pts)_ This should give you a ton of results! Maybe too many for the function to actually be useful in helping pick a wine. Perhaps there's some disagreement between the datasets. How should the `pairing`s and `ingredient`s from the scraped data and the Kaggle data each individually be pre-processed as text to obtain a good alignment?

_Response._

All the data shoubld have been converted to lower case and other errors include spelling errors should be taken care off.
Any punctuation errors should also be removed.